In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Fork-Join with Class Switching
model = Network('model')

In [3]:
# Create network
delay = Delay(model, 'Delay1')
delay2 = Delay(model, 'Delay2')
queue1 = Queue(model,'Queue1',SchedStrategy.PS)
queue2 = Queue(model,'Queue2',SchedStrategy.PS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)

jobclass1 = ClosedClass(model, 'class1', 10, delay, 0)
jobclass2 = ClosedClass(model, 'class2', 10, delay, 0)

In [4]:
# Service configurations
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(1.0))
delay.set_service(jobclass1, Exp(0.5))
delay.set_service(jobclass2, Exp(0.5))
delay2.set_service(jobclass2, Exp(2.0))

In [5]:
# Set routing matrix with class switching
P = model.init_routing_matrix()

# Class switching routing: jobclass1 -> jobclass2 -> jobclass1
P.set(jobclass1, jobclass2, delay, delay2, 1.0)    # Class switch: class1 -> class2 at delay
P.set(jobclass2, jobclass1, delay2, fork, 1.0)     # Class switch: class2 -> class1 at delay2

# Fork-join routing for class1
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [SolverJMT(model, seed=23000), SolverMVA(model)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.get_avg_table()

SOLVER 1: SolverJMT
JMT Model: /tmp/workspace/jsim/12088967178434273930/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.680398s.
  Station JobClass     QLen    Util    RespT  ResidT    ArvR    Tput
0  Delay1   class1   1.9093  1.9093   1.9791  1.9791  0.9831  0.9821
3  Delay2   class2   0.4864  0.4864   0.5077  0.5077  0.9821  0.9836
4  Queue1   class1  12.7339  0.9808  12.7957  6.3979  0.9836  0.9837
6  Queue2   class1   9.0597  0.9583   9.8528  4.9264  0.9836  0.9821
8    Join   class1  13.3251  0.0000   6.9336  6.9336  1.9492  0.9831
SOLVER 2: SolverMVA
MVA analysis [method: default(egflin), lang: java, env: 21.0.8] completed in 0.066085s.
  Station JobClass     QLen    Util    RespT  ResidT    ArvR    Tput
0  Delay1   class1   1.8658  1.8658   2.0000  2.0000  0.9329  0.9329
3  Delay2   class2   0.4665  0.4665   0.5000  0.5000  0.9329  0.9329
4  Queue1   class1  11.8695  0.9329  12.7229  6.3614  0.9329  0.9329
6  Queue2   class1  11.8695  0.9329  12.7229  6